In [1]:
import numpy as numpy
import scipy.io as spio
import scipy.io.wavfile as sciwav
import os.path as path
from os import listdir
from os.path import isfile, join
import os
import timeit as tt
import json
import sys
#import pandas
from tqdm import tqdm
import csv
from datetime import date
from datetime import datetime

In [2]:
# !pip install pyAudio

In [18]:
def getFilesInFloder(folderPath):
    onlyfiles = [f for f in listdir(folderPath) if isfile(join(folderPath, f)) 
                 and f.endswith("wav")]
    return onlyfiles

def getDirsInFolder(baseDirPath):
    onlySubDirs = [d for d in listdir(baseDirPath) if isdir(join(baseDirPath, d))]
    return onlySubDirs

def safe_wav_read(wav_file):
    try:
        std_sr = 16000
        sr, sig = wavio.read(wav_file)
        if sig.shape[0] < sig.size:
            sig = sig[0]
            print("\n{} is channel 2".format(wav_file))
        return sr, sig
    except:
        print("Error occured in read and convert wav to ndarray in file {}".format(wav_file))

In [19]:
# def get_filterbanks(nfilt=10,nfft=1024,samplerate=16000,lowfreq=0,highfreq=8000):
#     highfreq= highfreq or samplerate/2
#     assert highfreq <= samplerate/2, "highfreq is greater than samplerate/2"
#     # compute points evenly spaced in mels
#     lowmel = hz2mel_nature(lowfreq)
#     highmel = hz2mel_nature(highfreq)
#     melpoints = numpy.linspace(lowmel,highmel,nfilt+2)
#     # our points are in Hz, but we use fft bins, so we have to convert
#     #  from Hz to fft bin number
#     mid_freqs = mel2hz_nature(melpoints)
    
#     bins = numpy.floor((nfft+1)*mid_freqs/samplerate)
#     fbank = numpy.zeros([nfilt,nfft//2+1])
#     for j in range(0,nfilt):
#         for i in range(int(bins[j]), int(bins[j+1])):
#             fbank[j,i] = (i - bins[j]) / (bins[j+1]-bins[j])
#         for i in range(int(bins[j+1]), int(bins[j+2])):
#             fbank[j,i] = (bins[j+2]-i) / (bins[j+2]-bins[j+1])
#     print("Middel Frequences are {}".format(mid_freqs))
#     print("Bins are {}".format(bins))
#     return fbank

In [20]:
# get_filterbanks()

In [21]:
def preemphasis(signal, coeff=0.95):
    return numpy.append(signal[0], signal[1:] - coeff * signal[:-1])

def hz2mel_nature(freq):
    return 1127. * numpy.log(1. + freq / 700.)

def mel2hz_nature(mel):
    return 700. * (numpy.exp(mel / 1127.) - 1.)

def hz2mel(hz):
    return 2595 * numpy.log10(1+hz/700.)

def mel2hz(mel):
    return 700*(10**(mel/2595.0)-1)

In [22]:
# midfreq_array = numpy.array([16, 20, 26, 36, 48, 60, 80, 101, 256, 353])
# mel2hz_nature(midfreq_array)

In [23]:
def get_filterbanks_from_40(nfilt=40,nfft=1024,samplerate=16000,lowfreq=70,highfreq=8000):
    highfreq= highfreq or samplerate/2
    assert highfreq <= samplerate/2, "highfreq is greater than samplerate/2"

    # compute points evenly spaced in mels
    lowmel = hz2mel_nature(lowfreq)
    highmel = hz2mel_nature(highfreq)
    
    melpoints = numpy.linspace(lowmel,highmel,nfilt+2)
    mid_freqs = mel2hz_nature(melpoints)
#*********************************************
    c = 0
    for freq in mid_freqs:
        print("{}-{}".format("get_filterbanks_from_40",freq))
        c += 1 
    target_mid_freqs = numpy.empty(12,dtype=numpy.float)
    idx = 0
    for i in (2,3,5,6,8,9,10,12,22,32):
        print(mid_freqs[i])
        target_mid_freqs[idx] = mid_freqs[i]
        idx += 1
    nfilt = 10
#*********************************************
    bin = numpy.floor((nfft+1)*target_mid_freqs/samplerate)
    fbank = numpy.zeros([nfilt,nfft//2+1])
    for j in range(0,nfilt):
        for i in range(int(bin[j]), int(bin[j+1])):
            fbank[j,i] = (i - bin[j]) / (bin[j+1]-bin[j])
        for i in range(int(bin[j+1]), int(bin[j+2])):
            fbank[j,i] = (bin[j+2]-i) / (bin[j+2]-bin[j+1])
    return fbank

In [24]:
def get_filterbank_from_midfreqs(midFreqs,samplerate, n_filt, n_fft):
#     mid_freqs = midFreqs#[229.8,304.1,402.4,532.4,704.4,931.9,1233.1,1631.5,4000.,5500.]
    target_mid_freqs = numpy.empty(n_filt+2,dtype=numpy.float)
    idx = 0
    for freq in midFreqs:
        target_mid_freqs[idx] = freq
        idx += 1
#     target_mid_freqs[n_filt]=0.0
#     target_mid_freqs[n_filt+1]=0.0
    print(target_mid_freqs)
    bins = numpy.floor((n_fft+1)*target_mid_freqs/samplerate)
    print(len(bins))
    fbank = numpy.zeros([n_filt,n_fft//2+1])
    for j in range(0,n_filt):
        for i in range(int(bins[j]), int(bins[j+1])):
            fbank[j,i] = (i - bins[j]) / (bins[j+1]-bins[j])
        for i in range(int(bins[j+1]), int(bins[j+2])):
            fbank[j,i] = (bins[j+2]-i) / (bins[j+2]-bins[j+1])
    return fbank

In [25]:
def get_filterbanks(nfilt=10,nfft=1024,samplerate=16000,lowfreq=0,highfreq=8000):
    highfreq= highfreq or samplerate/2
    """Compute log Mel-filterbank energy features from an audio signal.
    :param signal: the audio signal from which to compute features. Should be an N*1 array
    :param samplerate: the sample rate of the signal we are working with, in Hz.
    :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    :param nfilt: the number of filters in the filterbank, default 26.
    :param nfft: the FFT size. Default is 512.
    :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
    :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
    :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97.
    :param winfunc: the analysis window to apply to each frame. By default no window is applied. You can use numpy window functions here e.g. winfunc=numpy.hamming
    :returns: A numpy array of size (NUMFRAMES by nfilt) containing features. Each row holds 1 feature vector.
    """
    assert highfreq <= samplerate/2, "highfreq is greater than samplerate/2"
    # compute points evenly spaced in mels
    lowmel = hz2mel_nature(lowfreq)
    highmel = hz2mel_nature(highfreq)
    melpoints = numpy.linspace(lowmel,highmel,nfilt+2)
    # our points are in Hz, but we use fft bins, so we have to convert
    #  from Hz to fft bin number
    mid_freqs = mel2hz_nature(melpoints)
    
    bins = numpy.floor((nfft+1)*mid_freqs/samplerate)
    fbank = numpy.zeros([nfilt,nfft//2+1])
    for j in range(0,nfilt):
        for i in range(int(bins[j]), int(bins[j+1])):
            fbank[j,i] = (i - bins[j]) / (bins[j+1]-bins[j])
        for i in range(int(bins[j+1]), int(bins[j+2])):
            fbank[j,i] = (bins[j+2]-i) / (bins[j+2]-bins[j+1])
    print("Middel Frequences are {}".format(mid_freqs))
    print("Bins are {}".format(bins))
    return fbank

loss_softmax_cross = label * (-1) * log(softmax_logits) + (1 — label) * (-1) * log(1 — softmax_logits)

In [26]:
def sigmoid(X):
    return 1/(1+numpy.exp(-X))

def relu(X):
    return numpy.maximum(0,X)

def softmax(X):
    expo = numpy.exp(X)
    expo_sum = numpy.sum(numpy.exp(X))
    return expo/expo_sum


In [27]:
def magspec(frames, NFFT):
    if numpy.shape(frames)[1] > NFFT:
        logging.warn(
            'frame length (%d) is greater than FFT size (%d), frame will be truncated. Increase NFFT to avoid.',
            numpy.shape(frames)[1], NFFT)
    complex_spec = numpy.fft.fft(frames, NFFT)
#     complex_spec = numpy.fft.rfft(frames, NFFT)
    return numpy.absolute(complex_spec)

def powspec(frames, NFFT):
    theFrames = magspec(frames,NFFT)
    return numpy.square(theFrames)
#     return 1.0 / NFFT * numpy.square(theFrames)

In [28]:
numpy.square(numpy.absolute([2-2j]))

array([8.])

In [29]:
"""
// input_test[] = raw data


void buffer_trans()
{
	int i,tmp_index;
	local_power=0;
	for (i = 0; i < fft_frame_size; i++){
		tmp_index = i*2;
		input[tmp_index] = input_test[i];
		input[tmp_index+1] = 0;
		local_power = local_power + (input_test[i]*input_test[i]);
	}
	local_power = sqrt(local_power);
	for (i = fft_frame_size*2; i < fft_order*2; i++)  {
		input[i] = 0;//zero padding
	}
}
"""
# def get_local_power

'\n// input_test[] = raw data\n\n\nvoid buffer_trans()\n{\n\tint i,tmp_index;\n\tlocal_power=0;\n\tfor (i = 0; i < fft_frame_size; i++){\n\t\ttmp_index = i*2;\n\t\tinput[tmp_index] = input_test[i];\n\t\tinput[tmp_index+1] = 0;\n\t\tlocal_power = local_power + (input_test[i]*input_test[i]);\n\t}\n\tlocal_power = sqrt(local_power);\n\tfor (i = fft_frame_size*2; i < fft_order*2; i++)  {\n\t\tinput[i] = 0;//zero padding\n\t}\n}\n'

In [30]:
frame_ms = 25
overlap_factor = 0.75
frames_per_segment = 4
num_fft = 1024
def runAccuracyTest(weights=None, wav_file=None, comp_base=1, threshold=-145,
                    filter_bank=None ,startPoint=1600, stepQuantity=400, record_csv=None):
    w1 = weights["w1"]
    w2 = weights["w2"]
    w3 = weights["w3"]
    b1 = weights["b1"]
    b2 = weights["b2"]
    b3 = weights["b3"]
    
    fs, sig = sciwav.read(wav_file)
#     print(sig.shape)
    if sig.shape[0] < sig.size:
        sig = sig.T[0]
    
    count_answer_1 = 0
    count_answer_0 = 0
    count_test_1 = 0
    count_test_0 = 0

    y_answer = numpy.empty([0,0],dtype=int)
    y_test = numpy.empty([0,0])
        
    tmp_data_list = []
    tmp_lbl_list = []
    data_length = len(sig)
    frame_size = int(fs * frame_ms // 1000)
    hop_step = int(frame_size - (frame_size*overlap_factor)) #for old overlap
    segment_size = int(frame_size * frames_per_segment)
    segment_overlap = int(segment_size * overlap_factor)
    segment_step = int(segment_size - segment_overlap)
    data_length = int(segment_step*(numpy.floor(data_length/segment_step)))+1
    sample_len = 513
    test_counter = 0
    csv_columns = ['test_no','time_sec','prediction','answer','is_speech','is_nonspeech','energy']
    csv_raw_list = []
    for i in range(segment_size, data_length, segment_step):
        
        test_counter += 1
        seg_2_sec = (test_counter*25)/1000
        record_dict = {}
        record_dict['test_no'] = test_counter
        record_dict['time_sec']= seg_2_sec
        idx1 = i-1600
        idx2 = i-1200
        idx3 = i-800
        idx4 = i-400
        
        s1 = sig[idx1:idx2] #0-399
        s2 = sig[idx2:idx3] #400-799run_test_main,number
        s3 = sig[idx3:idx4] #800-1199
        s4 = sig[idx4:i]    #1200-1599
        
#         s1 = s1*numpy.hanning(len(s1))
#         s2 = s2*numpy.hanning(len(s2))
#         s3 = s3*numpy.hanning(len(s3))
#         s4 = s4*numpy.hanning(len(s4))
        
        s1 = s1.reshape(1,len(s1))
        s2 = s2.reshape(1,len(s2))
        s3 = s3.reshape(1,len(s3))
        s4 = s4.reshape(1,len(s4))
        
        s1 = powspec(s1,num_fft)
        s2 = powspec(s2,num_fft)
        s3 = powspec(s3,num_fft)
        s4 = powspec(s4,num_fft)
        
        s1 = numpy.split(s1.T,[0,sample_len],axis=0)[1]
        s2 = numpy.split(s2.T,[0,sample_len],axis=0)[1]
        s3 = numpy.split(s3.T,[0,sample_len],axis=0)[1]
        s4 = numpy.split(s4.T,[0,sample_len],axis=0)[1]
#         print("s1 is {}".format(s1))
#         return
#         feat1 = numpy.dot(s1,aFBank.T) # compute the filterbank energies
        feat1  = numpy.matmul(aFBank,s1)
        feat1 = numpy.where(feat1 == 0,numpy.finfo(float).eps,feat1) # if feat is
        feat1 = numpy.log(feat1)
#         feat1 = 10 * numpy.log10(feat1)
        
#         feat2 = numpy.dot(s2,aFBank.T) # compute the filterbank energies
        feat2  = numpy.matmul(aFBank,s2)
        feat2 = numpy.where(feat2 == 0,numpy.finfo(float).eps,feat2) # if feat is 
        feat2 = numpy.log(feat2)
#         feat2 = 10 * numpy.log10(feat2)
        
#         feat3 = numpy.dot(s3,aFBank.T) # compute the filterbank energies
        feat3  = numpy.matmul(aFBank,s3)
        feat3 = numpy.where(feat3 == 0,numpy.finfo(float).eps,feat3) # if feat is 
        feat3 = numpy.log(feat3)
#         feat3 = 10 * numpy.log10(feat3)
        
#         feat4 = numpy.dot(s4,aFBank.T) # compute the filterbank energies
        feat4  = numpy.matmul(aFBank,s4)
        feat4 = numpy.where(feat4 == 0,numpy.finfo(float).eps,feat4) # if feat is 
        feat4 = numpy.log(feat4)
#         feat4 = 10 * numpy.log10(feat4)
        
#         x= numpy.array([feat1[0],feat2[0],feat3[0],feat4[0]]).reshape(1,40)
        x= numpy.array([feat1,feat2,feat3,feat4]).reshape(1,40)
        x_40 = x[0:40]
        
        max_ele = numpy.amax(x_40,axis=1)
        min_ele = numpy.amin(x_40,axis=1)
        
        # normalize
        x_normalize = (x_40-min_ele)/(max_ele-min_ele+0.0001)
        sum_of_x = numpy.sum(x_normalize)
        record_dict['energy']=sum_of_x
        #counting the test
        #performing the model weight mulplications
        answer = softmax(numpy.matmul(relu(numpy.matmul(relu(numpy.matmul(x_normalize,w1)+b1),w2)+b2),w3)+b3)
        record_dict['is_speech'] = answer[0,1]
        record_dict['is_nonspeech'] = answer[0,0]
        if sum_of_x > threshold:
            if answer[0,0] > answer[0,1]:
                y_test = numpy.append(y_test, 0)
                count_test_0 = count_test_0+1
                record_dict['prediction'] = 0
            else:
                y_test = numpy.append(y_test, 1)
                count_test_1 += 1
                record_dict['prediction'] = 1
        else:
            y_test = numpy.append(y_test, 0)
            count_test_0 += 1
            record_dict['prediction'] = 0
            
        #counting our answer
        if comp_base == 1:
            if sum_of_x > threshold:
                y_answer = numpy.append(y_answer,1)
                count_answer_1 = count_answer_1+1
                record_dict['answer'] = 1
            else:
                y_answer = numpy.append(y_answer,0)
                count_answer_0 += 1
                record_dict['answer'] = 0
        else:
            y_answer = numpy.append(y_answer,0)
            count_answer_0 += 1
            record_dict['answer'] = 0
        csv_raw_list.append(record_dict)
        
    acc = 0
    LED = 0
    wrong = numpy.empty([0,0])
    y_answer_len = len(y_answer)    
    y_test_len = len(y_test)
    
    if comp_base == 1: #if our test data is human voice
        for c in range(y_answer_len):
            acc += abs(y_answer[c]-y_test[c])

        acc = (y_answer_len-acc)/y_answer_len
    else:
        acc = count_test_0/(count_test_0+count_test_1)
    
    with open(record_csv,"w") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in csv_raw_list:
            writer.writerow(data)
    
    return acc #json.dumps(loop_record,cls=NumpyEncoder)
        
    

In [31]:
numpy.set_printoptions(threshold=sys.maxsize)
def dumpFB_Array(midfreqs):
    theFB = get_filterbank_from_midfreqs(midfreqs,16000,10,1024)
    rows = theFB.shape[0]
    cols = theFB.shape[1]
    with open("./outputfb/mel_fb_20200717.h","w")as f:
        f.write("/* middle frequences:{} */\n".format(midfreqs))
        f.write("const float fbarray[{}][{}]=".format(rows,cols))
        f.write("{")
        for i in range(rows):
            f.write("{")
            for j in range(cols):
                f.write(str(theFB[i][j]))
                f.write(",")
            f.write("},")
            f.write("\n")
        f.write("}")
        

In [32]:
dumpFB_Array([0.0,195.0985245,251.8401972,312.1788118,376.342384,444.5733837,517.1296516,1550.447293,2554.078667,3461.030019,4620.759758,8000.0])
# 80/20 : 0.0,251.8,312.1,376.3,444.5,517.1,594.2,676.3,763.5,1550.4,2979.7,8000.0
# 70/30 : 0.0,312.17,376.34,444.57,517.12,676.33,763.57,856.35,1550.44,2979.71,3212.98,8000.0
# 60/40 : 0.0,312.17,376.34,444.57,517.12,676.33,763.57,1550.44,2979.71,3212.98,4303.57,8000.0
# 50/50 : 0.0,312.17,376.34,444.57,517.12,676.33,1550.44,2979.71,3212.98,4303.57,5316.72,,8000.0


[   0.         195.0985245  251.8401972  312.1788118  376.342384
  444.5733837  517.1296516 1550.447293  2554.078667  3461.030019
 4620.759758  8000.       ]
12


In [38]:
#NCTU ICFILTER Middle Frequences
# Testing need to modify two variables: __midfreqs, weight_mat_file
# __midfreqs = [0.0,444.5,594.2,676.3,763.5,856.3,955.018,1171.497,1290.134,1416.292,1550.44,8000.0]
__midfreqs = [0.0,195.0985245,251.8401972,312.1788118,376.342384,444.5733837,517.1296516,1550.447293,2554.078667,3461.030019,4620.759758,8000.0]
__midfreqs_array = numpy.array(__midfreqs)
aFBank = get_filterbank_from_midfreqs(__midfreqs_array,16000,10,1024)#[:,1:]
print("aFBank shape is {}".format(aFBank.shape))
dumpFB_Array(__midfreqs_array)
freq_ratio = 64
# aFBank = get_filterbanks()
def MultiFileTest():
    weight_mat_file = "../../Weights/vad_weight_128x32_20200721_00_15_50.mat"
    loaded_weights = spio.loadmat(weight_mat_file)
    #get date and time string:
    dateStr = "{}".format(str(date.today()).replace("-", ""))
    timeStr = "{}".format(str(datetime.now().strftime("%H_%M_%S")))
    csv_output_dir = "./TestReports/Test_Float_32_ln_{}_freqs_{}_{}/".format(freq_ratio,dateStr,timeStr)
    if not path.exists(csv_output_dir):
        os.makedirs(csv_output_dir)
        
    test_speech_dir = "../Drawing/compared_wav/"
    test_speech_files = getFilesInFloder(test_speech_dir)
    test_speech_acc = []
    flistlen = len(test_speech_files)
    for f in tqdm(test_speech_files):
        speech_csv_file_name = csv_output_dir+"Float32W_Test_32_ln_{}_{}_Speech_{}_{}.csv".format(freq_ratio,f,
                                                                                          dateStr,timeStr)
        _acc=0
        _acc = runAccuracyTest(loaded_weights, wav_file=test_speech_dir+f, comp_base=1, 
                                               threshold=-145, filter_bank=aFBank ,startPoint=1600, 
                                               stepQuantity=400,record_csv=speech_csv_file_name)
        print("speech file {} acc value is:{}".format(f,_acc))
        test_speech_acc.append(_acc)
    speech_sum_of_acc = sum(test_speech_acc)
    speech_avg_acc = speech_sum_of_acc/flistlen
    print("\nThe average accuarncy of testing {} speech files is {}\n".format(flistlen,speech_avg_acc))
    ##########################################################################
#     test_noise_dir = "../../TestWavFiles/Noise/20200717/"
#     test_noise_dir = "../MainSrc/Drawing/raw_wav/"
#     test_noise_files = getFilesInFloder(test_noise_dir)
#     test_noise_acc = []
#     flistlen2 = len(test_noise_files)
#     for f2 in tqdm(test_noise_files):
#         print("current processing file is {}".format(f2))
#         noise_csv_file_name = csv_output_dir+"Float32W_Test_32_ln_{}_{}_Noise_{}_{}.csv".format(freq_ratio,f2,
#                                                                                         dateStr,timeStr)
#         _acc2= 0.
#         _acc2 = runAccuracyTest(loaded_weights, wav_file=test_noise_dir+f2, comp_base=0, 
#                                                threshold=-145, filter_bank=aFBank ,startPoint=1600, 
#                                                stepQuantity=400,record_csv=noise_csv_file_name)
#         print("noise file {} acc value is:{}".format(f2, _acc2))
#         test_noise_acc.append(_acc2)
#     noise_sum_of_acc = sum(test_noise_acc)
#     noise_avg_acc = noise_sum_of_acc/flistlen2
#     print("\nThe average accuarncy of testing {} noise files is {}\n".format(flistlen2,noise_avg_acc))
    

[   0.         195.0985245  251.8401972  312.1788118  376.342384
  444.5733837  517.1296516 1550.447293  2554.078667  3461.030019
 4620.759758  8000.       ]
12
aFBank shape is (10, 513)
[   0.         195.0985245  251.8401972  312.1788118  376.342384
  444.5733837  517.1296516 1550.447293  2554.078667  3461.030019
 4620.759758  8000.       ]
12


In [39]:
MultiFileTest()

 33%|███▎      | 1/3 [00:00<00:00,  3.86it/s]

speech file 1016拍木板隔間.dat.wav acc value is:0.16624685138539042


 67%|██████▋   | 2/3 [00:01<00:00,  2.36it/s]

speech file Speech_CL.mp3.wav acc value is:0.9589442815249267


100%|██████████| 3/3 [00:01<00:00,  2.29it/s]

speech file Sax.dat.wav acc value is:0.0025188916876574307

The average accuarncy of testing 3 speech files is 0.3759033415326582



In [35]:
def singleFileTest():
    weight_mat_file = "./Weights/weight_128x32_20200427_17_27_48_norm.mat"
    loaded_weights = spio.loadmat(weight_mat_file)
    #get date and time string:
    dateStr = "{}".format(str(date.today()).replace("-", ""))
    timeStr = "{}".format(str(datetime.now().strftime("%H_%M_%S")))
    #create csv report saving directory
    csv_output_dir = "./TestReports/Test_Int32_{}_{}/".format(dateStr,timeStr)
    if not path.exists(csv_output_dir):
        os.makedirs(csv_output_dir)
    #Testing Speech
    f = "BL.mp3.wav"
    test_wav = "./TestWavFiles/Speech/"+f   
    cflag = 1
    print("Starting VAD Test")
    csv_file_name = csv_output_dir+"Int32_Test_In_MelSpec_{}_CleanSpeech_{}_{}.csv".format(f, dateStr,timeStr)
    ret_acc = runAccuracyTest(loaded_weights, wav_file=test_wav, comp_base=cflag, threshold=16, 
                              filter_bank=aFBank ,startPoint=1600, stepQuantity=400, 
                              record_csv=csv_file_name)
        

In [20]:
# singleFileTest()